# Desarrollo de un sistema de diagnóstico de enfermedades en hojas de tomate mediante PlantCV y modelos de aprendizaje profundo

La detección temprana de enfermedades en cultivos es vital para la agricultura sostenible. Utilizando PlantCV (https://plantcv.readthedocs.io/), una biblioteca enfocada en la visión por computadora para plantas, junto con TensorFlow o PyTorch, el alumno se encargará de crear modelos de clasificación multiclase robustos. Estos modelos tendrán como objetivo identificar 10 distintas enfermedades y una clase de salud en hojas de tomate. Para el entrenamiento y evaluación, se usará el Tomato Leaves Dataset de Kaggle (https://www.kaggle.com/datasets/ashishmotwani/tomato), que contiene más de 20,000 imágenes recogidas en entornos de laboratorio e in situ. Dada la naturaleza práctica de la aplicación, se buscará desarrollar un modelo lo suficientemente eficiente como para poder ser implementado en aplicaciones móviles, permitiendo diagnósticos ágiles y efectivos en el campo.

## Carga de la data

In [1]:
# codigo para cargar la data y prepararla
from src.utils.data_loader import get_data_loaders

data_dir = "dataset_lite"
train_loader, valid_loader, class_names = get_data_loaders(data_dir)

print(f"Clases detectadas: {class_names}")


Clases detectadas: ['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'healthy', 'powdery_mildew']


## Exploracion de los datos

In [ ]:
# codigo para explorar caracteristicas escenciales, con plantCV y demas librerias
# para ver tamanos, balance de train y validacion, separar un poco para el test

## Modelos de clasificacion multiclases

In [5]:
!nvidia-smi

Thu Feb 13 10:03:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   46C    P8              2W /   80W |     548MiB /   8188MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Opcional para entrenar con la GPU
!pip install torch==2.1.0+cu121 torchvision==0.16.0+cu121 --index-url https://download.pytorch.org/whl/cu121 --timeout 1000 --retries 10

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.1.0%2Bcu121-cp310-cp310-win_amd64.whl (2473.9 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.16.0%2Bcu121-cp310-cp310-win_amd64.whl (5.8 MB)
     ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
     ----- ---------------------------------- 0.8/6.2 MB 1.3 MB/s eta 0:00:05
     ----------- ---------------------------- 1.8/6.2 MB 2.2 MB/s eta 0:00:02
     -------------------- ------------------- 3.1/6.2 MB 3.2 MB/s eta 0:00:01
     ------------------------------ --------- 4.7/6.2 MB 4.0 MB/s eta 0:00:01
     -------------------------------------- - 6.0/6.2 MB 4.6 MB/s eta 0:00:01
     ---------------------------------------- 6.2/6.2 MB 4.4 MB/s eta 0:00:00
     ---------------------------

In [2]:
# Se crean los modelos y se los entrenan
from src.models.model import TomatoDiseaseClassifier

# Número de clases detectadas en el dataset
num_classes = len(class_names)

# Inicializar el modelo
model = TomatoDiseaseClassifier(num_classes)
print(model)


TomatoDiseaseClassifier(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=11, bias=True)
)


In [3]:
#pip install torch==2.1.0+cu121 torchvision==0.16.0+cu121 --index-url https://download.pytorch.org/whl/cu121 --timeout 1000 --retries 10
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"{device_count} GPU(s) available:")
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA compatible GPU detected.")


1 GPU(s) available:
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
from src.models.train import train_model

# Definir hiperparámetros
num_epochs = 20
learning_rate = 0.001

# Entrenar el modelo
trained_model = train_model(model, train_loader, valid_loader, num_epochs, learning_rate)

Epoch 1/20 - Loss: 0.0626 - Accuracy: 46.73%
Epoch 2/20 - Loss: 0.0398 - Accuracy: 54.09%
Epoch 3/20 - Loss: 0.0057 - Accuracy: 54.55%
Epoch 4/20 - Loss: 0.0039 - Accuracy: 54.27%
Epoch 5/20 - Loss: 0.0006 - Accuracy: 55.00%
Epoch 6/20 - Loss: 0.0003 - Accuracy: 56.36%
Epoch 7/20 - Loss: 0.0002 - Accuracy: 56.09%
Epoch 8/20 - Loss: 0.0002 - Accuracy: 56.00%
Epoch 9/20 - Loss: 0.0002 - Accuracy: 55.82%
Epoch 10/20 - Loss: 0.0001 - Accuracy: 55.73%
Epoch 11/20 - Loss: 0.0001 - Accuracy: 55.55%
Epoch 12/20 - Loss: 0.0001 - Accuracy: 55.91%
Epoch 13/20 - Loss: 0.0000 - Accuracy: 56.00%
Epoch 14/20 - Loss: 0.0000 - Accuracy: 56.18%
Epoch 15/20 - Loss: 0.0000 - Accuracy: 56.45%
Epoch 16/20 - Loss: 0.0000 - Accuracy: 56.64%
Epoch 17/20 - Loss: 0.0000 - Accuracy: 56.45%
Epoch 18/20 - Loss: 0.0000 - Accuracy: 56.45%
Epoch 19/20 - Loss: 0.0000 - Accuracy: 56.45%
Epoch 20/20 - Loss: 0.0000 - Accuracy: 56.82%


## Comparacion de los mejores modelos obtenidos

In [ ]:
# Se compara los modelos obtenidos y se decide por el mejor

## Predicciones con el mejor modelo para este caso

In [ ]:
#Se crea una clase para hacer la prediccion en base a una imagen nueva